In [43]:
import json
import sys
sys.path.append('/home/nbuser/library')
import time
import pandas as pd
import requests

In [44]:
def get_latlon(address):
    ''' Takes an address as an input, returns latitude and longitude coordinates. '''
    
    time.sleep(0.2)
    
    url = 'https://us1.locationiq.com/v1/search.php'

    params = {
        'key': '3e10ca48173365',
        'q': address,
        'format': 'json',
    }

    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        return pd.Series({'Latitude': response.status_code, 'Longitude': response.status_code})

    lat = response.json()[0]['lat']
    lon = response.json()[0]['lon']
    
    return pd.Series({'Latitude': lat, 'Longitude': lon})

In [45]:
get_latlon('26 Symonds Road, Auckland')

Latitude     -36.853934
Longitude    174.768672
dtype: object

In [20]:
#### Finding lat and lon for every property in file

In [46]:
df = pd.read_csv('Properties.csv')

In [47]:
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park


In [48]:
df = df.drop(columns=['Latitude','Longitude','SA1'])

In [49]:
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,48,27,24,21,24,21,Manurewa
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,42,18,12,21,15,30,Karaka
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,42,18,12,21,15,30,Karaka
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,42,6,21,21,12,15,Onehunga
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,93,27,33,30,21,33,Clover Park


In [51]:
df[['Latitude','Longitude']] = df.Address.apply(get_latlon)

In [52]:
def combine_lat_lon(lat, lon):
    return (lat, lon)

In [53]:
df['Combined coordinates'] = df.apply(lambda x : combine_lat_lon(x['Latitude'], x['Longitude']), axis = 1)

In [54]:
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,Latitude,Longitude,Combined coordinates
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,48,27,24,21,24,21,Manurewa,-37.012914,174.903973,"(-37.012914, 174.903973)"
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,42,18,12,21,15,30,Karaka,-37.0636507,174.9228185,"(-37.0636507, 174.9228185)"
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,42,18,12,21,15,30,Karaka,429,429,"(429, 429)"
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,42,6,21,21,12,15,Onehunga,-36.91325,174.7874144,"(-36.91325, 174.7874144)"
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,93,27,33,30,21,33,Clover Park,-36.9789324,174.8926273,"(-36.9789324, 174.8926273)"
